In [1]:
import numpy as np
import binascii

In [2]:
def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)

def int2bytes(i):
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))

def string2bits(s=''):
    return [bin(ord(x))[2:].zfill(8) for x in s]

def bits2string(b=None):
    return ''.join([chr(int(x, 2)) for x in b])

In [3]:
def decimalToBinary(num , places):
    return ('{0:0' +  str(places) + 'b}').format(num)


def binaryToDecimal(binary):
    return int(binary, 2)
        
def xorBinary(binary,key = "00000000"):
    i=0
    encBin = ""
    while(i<8):
        if(binary[i] == '0' and key[i] == '0') or (binary[i] == '1' and key[i] == '1'):
            encBin+="0"
        else:
            encBin+="1"
        i+=1
        
#     print(binary , key , encBin)    
    return encBin
    
def compressBinary(binary):
    i=0
    compressBin = ""
    while(i<8):
        if(i%2 == 0):
            if((binary[i] == '0' and binary[i+1] == '0') or (binary[i] == '1' and binary[i+1] == '1')):
                compressBin+="0"
            else:
                compressBin+="1"
        i+=1
        
    return compressBin

def to_twoscomplement(bits, value):
    if value < 0:
        value = ( 1<<bits ) + value
    formatstring = '{:0%ib}' % bits
    return formatstring.format(value)

# Code for decoding

In [4]:
import sys, traceback
import time , socket
import struct

In [5]:
def server_program():
    # get the hostname
    host = 'localhost'
    port = 3000  # initiate port no above 1024
    count = 0
    current = 0
    currentByte = ''
    last = 0
    org_data = []
    start_time = None
    end_time = None

    
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # get instance
    server_socket.bind((host, port))  # bind host address and port together
    server_socket.listen(1)

    try :
        connection,client_address = server_socket.accept()
        while True:
            data = connection.recv(1)
            if not data:
                end_time = int(time.time() * 1000)
                print("Time in MS :" , end_time - start_time)
                print("Packets Transfered :" , count)
                connection.close()
                server_socket.close()
                return count , (end_time - start_time)
            
            data = decimalToBinary(int.from_bytes(data, byteorder='little'), 8)
            if data[0] == '0':
                if last == 0:
                    start_time = int(time.time() * 1000)
                count+=1
                current+=1
                currentByte+=data[1:]
                if current == 3:
                    dataPoint = int(currentByte[1:],2)
                    if currentByte[0] == '1':
                        dataPoint *= -1
                    org_data.append(dataPoint)
                    last=dataPoint
                    current = 0
                    currentByte = ''
                    
            else:
                count+=1
                dataPoint = int(data[2:],2)
                if data[1]=='1':
                    dataPoint*=-1
                dataPoint+=last
                org_data.append(dataPoint)
                last = dataPoint
        
    except Exception:
        print("Some Error Occured")
        traceback.print_exc()
        connection.close()
        server_socket.close()

In [20]:
server_program()

Time in MS : 2004
Packets Transfered : 980117


(980117, 2004)

In [15]:
di = {}

for i in range(15):
    packets, ttime = server_program()
    if(di.get(packets)) :
        di[packets].append(ttime)
    else :
        di[packets] = [ttime]
    time.sleep(3)

Time in MS : 3053
Packets Transfered : 980117
Time in MS : 2129
Packets Transfered : 980117
Time in MS : 2052
Packets Transfered : 980117
Time in MS : 1165
Packets Transfered : 490058
Time in MS : 1110
Packets Transfered : 490058
Time in MS : 1367
Packets Transfered : 490058
Time in MS : 860
Packets Transfered : 245029
Time in MS : 808
Packets Transfered : 245029
Time in MS : 1496
Packets Transfered : 245029
Time in MS : 1458
Packets Transfered : 122514
Time in MS : 320
Packets Transfered : 122514
Time in MS : 347
Packets Transfered : 122514
Time in MS : 944
Packets Transfered : 61257
Time in MS : 234
Packets Transfered : 61257
Time in MS : 351
Packets Transfered : 61257


# Code for Un-compressed

In [8]:
def UC_server_program():
    # get the hostname
    host = 'localhost'
    port = 6000  # initiate port no above 1024
    count = 0
    start = 0
    data_byte = ''
    org_data = []
    start_time = None
    end_time = None

    
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # get instance
    server_socket.bind((host, port))  # bind host address and port together
    server_socket.listen(1)

    try :
        connection,client_address = server_socket.accept()
        while True:
            data = connection.recv(1)
            if not data:
                end_time = int(time.time() * 1000)
                print("Time in MS :" , end_time - start_time)
                print("Packet Transfered :" , count)
                connection.close()
                server_socket.close()
                return count , (end_time - start_time)
            
            else:
                if start == 0:
                    start_time = int(time.time() * 1000)
                    start = 1
                count+=1
                data = int.from_bytes(data, byteorder = 'little')
                data_byte+= decimalToBinary(data,8)
                if count%3 == 0:
                    data = binaryToDecimal(data_byte)
                    org_data.append(data)
                    data_byte = ''

    except Exception:
        print("Some Error Occured")
        traceback.print_exc()
        connection.close()
        server_socket.close()

In [9]:
UC_server_program()

Time in MS : 5282
Packet Transfered : 2786973


(2786973, 5282)

In [10]:
di2 = {}

for i in range(15):
    packets, ttime = UC_server_program()
    if(di2.get(packets)) :
        di2[packets].append(ttime)
    else :
        di2[packets] = [ttime]
    time.sleep(3)

Time in MS : 1155
Packet Transfered : 928991
Time in MS : 1087
Packet Transfered : 928991
Time in MS : 1111
Packet Transfered : 928991
Time in MS : 555
Packet Transfered : 464495
Time in MS : 551
Packet Transfered : 464495
Time in MS : 545
Packet Transfered : 464495
Time in MS : 279
Packet Transfered : 232247
Time in MS : 284
Packet Transfered : 232247
Time in MS : 279
Packet Transfered : 232247
Time in MS : 166
Packet Transfered : 116123
Time in MS : 146
Packet Transfered : 116123
Time in MS : 145
Packet Transfered : 116123
Time in MS : 81
Packet Transfered : 58061
Time in MS : 80
Packet Transfered : 58061
Time in MS : 81
Packet Transfered : 58061
